# train_model.py

In this file, I:

1. **Load** GitHub PR data from `github_prs_raw.json`.
2. **Flatten** the JSON using `pandas.json_normalize`.
3. **Clean & Engineer** features (like `time_to_merge_hours`, label counts, etc.).
4. **Train & Evaluate** a Random Forest model.
5. **Save** the model (`pr_time_model.joblib`).

I can then run `python train_model.py` whenever I want to retrain or update the model.

In [2]:
import sys
print(sys.executable)



/Users/cememirsenyurt/opt/anaconda3/envs/apress/bin/python
